In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Connect on Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Import the meetlogs file
downloaded = drive.CreateFile({'id':"15Pq0ZcNZ7oCn72qiJDIBofVePWXR3hRV"})
downloaded.GetContentFile('meetlogs.csv')

In [ ]:
import pandas as pd

# Read the csv file using pandas library
data = pd.read_csv('meetlogs.csv')
data.head()

,Date,Event Name,Event Description,Meeting Code,Participant Identifier,Participant Outside Organisation,Client Type,Organizer Email,Product Type,Duration,Call Rating out of 5,Participant Name,IP Address,City,Country,Network Round Trip Time Mean In ms,Transport Protocol,Estimated Upload Bandwidth in kbps,Estimated Download Bandwidth in kbps,Audio Receive Packet Loss Max,Audio Receive Packet Loss Mean,Audio Receive Duration,Audio Send Bitrate Mean in kbps,Audio Send Packet Loss Max,Audio Send Packet Loss Mean,Audio Send Duration,Calendar Event Id,Conference ID,Network Recv Jitter Mean in ms,Network Recv Jitter Max in ms,Network Send Jitter Mean in ms,Screencast Receive Bitrate Mean in kbps,Screencast Receive FPS Mean,Screencast Receive Long Side Median,Screencast Receive Packet Loss Max,Screencast Receive Packet Loss Mean,Screencast Receive Duration,Screencast Receive Short Side Median,Screencast Send Bitrate Mean in kbps,Screencast Send FPS Mean,Screencast Send Long Side Median,Screencast Send Packet Loss Max,Screencast Send Packet Loss Mean,Screencast Send Duration,Screencast Send Short Side Median,Video Receive FPS Mean,Video Receive Long Side Median,Video Receive Packet Loss Max,Video Receive Packet Loss Mean,Video Receive Duration,Video Receive Short Side Median,Network Congestion Ratio,Video Send Bitrate Mean in kbps,Video Send FPS Mean,Video Send Long Side Median,Video Send Packet Loss Max,Video Send Packet Loss Mean,Video Send Duration,Video Send Short Side Median,Action Reason,Action Description,Target Display Names,Target Email,Target Phone Number
0,"Sep 17, 2021, 12:05:24 PM GMT-4",Endpoint left,The endpoint left a video meeting,YYQBSKRPZV,mpmukono@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,3875.0,NaN,Mukono Uganda,154.231.51.110,Kampala,UG,103.0,UDP,859.0,307.0,6.0,0.0,3874,16.0,3.0,0.0,3884,f9d3jha4l01dun1qpcs5573as1_20210917T150000Z,wVt-fJuUbcN5sT4goBZ_YKII,9.0,86.0,14.0,131.0,11.0,1440.0,0.0,0.0,3818,772.0,NaN,NaN,NaN,NaN,NaN,0,NaN,19.0,640.0,0.0,0.0,3131,360.0,0,818.0,7.0,640.0,1.0,0.0,3884,360.0,NaN,NaN,NaN,NaN,NaN
1,"Sep 17, 2021, 12:05:24 PM GMT-4",Endpoint left,The endpoint left a video meeting,YYQBSKRPZV,areebah.ahmed@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,3879.0,NaN,Areebah Ahmed,2601:5c1:4402:9c0:8ac:6e34:5977:bc98,Richmond,US,38.0,UDP,145.0,768.0,13.0,0.0,1164,22.0,4.0,0.0,3583,f9d3jha4l01dun1qpcs5573as1_20210917T150000Z,wVt-fJuUbcN5sT4goBZ_YKII,13.0,89.0,6.0,NaN,NaN,NaN,NaN,NaN,0,NaN,217.0,13.0,1440.0,29.0,0.0,2407,772.0,7.0,640.0,0.0,0.0,1162,360.0,0,134.0,20.0,320.0,23.0,0.0,1020,180.0,NaN,NaN,NaN,NaN,NaN
2,"Sep 17, 2021, 12:01:27 PM GMT-4",Endpoint left,The endpoint left a video meeting,FVEAZBZPIY,mpkadzakalowa@villagebookbuilders.org,No,Web,mentor@villagebookbuilders.org,Google Meet,4148.0,NaN,Kadzakalowa Malawi,105.234.162.3,Blantyre,MW,70.0,UDP,548.0,645.0,1.0,0.0,3659,22.0,6.0,0.0,4087,f20m133iihao3mivi992s1hs28_20210917T150000Z,uru1tOKi63NYOC5Lh4eTYKII,13.0,26.0,6.0,285.0,5.0,1440.0,0.0,0.0,3307,900.0,NaN,NaN,NaN,NaN,NaN,0,NaN,21.0,640.0,0.0,0.0,3595,360.0,1,509.0,15.0,640.0,6.0,0.0,4087,360.0,NaN,NaN,NaN,NaN,NaN
3,"Sep 17, 2021, 12:00:31 PM GMT-4",Endpoint left,The endpoint left a video meeting,FVEAZBZPIY,isuri.dissanayake@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,3591.0,NaN,isuri dissanayake,212.104.224.93,Colombo,LK,297.0,UDP,583.0,353.0,10.0,0.0,1756,34.0,79.0,0.0,1756,f20m133iihao3mivi992s1hs28_20210917T150000Z,uru1tOKi63NYOC5Lh4eTYKII,24.0,187.0,11.0,NaN,NaN,NaN,NaN,NaN,0,NaN,309.0,5.0,1440.0,7.0,0.0,1538,900.0,14.0,640.0,17.0,1.0,1756,360.0,5,536.0,23.0,640.0,27.0,0.0,1756,360.0,NaN,NaN,NaN,NaN,NaN
4,"Sep 17, 2021, 11:57:50 AM GMT-4",Endpoint left,The endpoint left a video meeting,PGSABQXGOZ,jacqueline.floresss17@gmail.com,Yes,Web,brittany.a@villagebookbuilders.org,Google Meet,1025.0,NaN,Jacqueline Flores,NaN,NaN,NaN,76.0,UDP,923.0,286.0,1.0,0.0,1021,6.0,NaN,NaN,1021,rbfcrcpna9ej5qok872mr275ls,IFAzibi6mqVNuY

In [ ]:
# Select the columns of interest 
data = data[['Participant Identifier', 'Participant Name', 'Organizer Email', 'City', 'Country']]
# Select only rows where the Organizer Email is 'mentor@villagebookbuilders.org' (the remainings are supposed not to be related to the mentoring program) and drop the column
data_mp = data.loc[data['Organizer Email'] == 'mentor@villagebookbuilders.org']
data_mp = data_mp.drop(columns=['Organizer Email'])
# Find all the mentees (whose either Identifier starts with 'mp' or Name with 'Computer') et discard them from the dataframe
data_mentors = data_mp.loc[~(data['Participant Identifier'].str.startswith('mp', na=False) | data['Participant Name'].str.startswith('Computer', na=False))]
# Drop all duplicates from the Participant Identifier
data_mentors = data_mentors.drop_duplicates(subset=['Participant Identifier'])

In [ ]:
data_mentors

,Participant Identifier,Participant Name,City,Country
1,areebah.ahmed@villagementors.org,Areebah Ahmed,Richmond,US
3,isuri.dissanayake@villagementors.org,isuri dissanayake,Colombo,LK
7,jenny.yang@villagementors.org,Jenny Yang,Druid Hills,US
11,sandy.granger@villagementors.org,Sandy Granger,Burien,US
13,samantha.yong@villagementors.org,samantha yong,Kuala Lumpur,MY
...,...,...,...,...
14714,katrinamaktaz@gmail.com,Katrina Maktaz,NaN,NaN
14725,ram.velamuri@villagementors.org,Ram Velamuri,Bengaluru,IN
14896,alondra.ramirez0215@gmail.com,Alondra Ramirez Herrera,NaN,NaN
14958,satheesh.tamilselvan@villagementors.org,satheesh tamilselvan,Urbana,US


In [ ]:
# Convert filtered dataframe to csv file
data_mentors.to_csv('mentors.csv', index=False)

In [ ]:
# Install some libraries
!pip install pycountry-convert
!pip install geopy
!pip install folium

     |████████████████████████████████| 10.1 MB 34.1 MB/s 
     |████████████████████████████████| 280 kB 46.5 MB/s 
     |████████████████████████████████| 213 kB 66.1 MB/s 
     |████████████████████████████████| 280 kB 72.3 MB/s 
     |████████████████████████████████| 280 kB 72.5 MB/s 
     |████████████████████████████████| 280 kB 67.8 MB/s 
     |████████████████████████████████| 279 kB 44.9 MB/s 
     |████████████████████████████████| 279 kB 57.3 MB/s 
     |████████████████████████████████| 272 kB 57.2 MB/s 
     |████████████████████████████████| 272 kB 55.6 MB/s 
     |████████████████████████████████| 272 kB 55.5 MB/s 
     |████████████████████████████████| 270 kB 56.3 MB/s 
     |████████████████████████████████| 270 kB 52.4 MB/s 
     |████████████████████████████████| 270 kB 55.3 MB/s 
     |████████████████████████████████| 248 kB 59.5 MB/s 
     |████████████████████████████████| 247 kB 58.5 MB/s 
     |████████████████████████████████| 246 kB 59.9 MB/s 
     |███████

In [ ]:
from pycountry_convert import country_alpha2_to_country_name

# Create new dataframe where we count the number of mentors per country
data_location = data_mentors['Country'].value_counts()
data_grouped = data_location.to_frame()
data_grouped.reset_index(level=0, inplace=True)
data_grouped = data_grouped.rename(columns={"index": "Country", "Country": "count"})
data_grouped["Country Fullname"] = data_grouped["Country"].apply(country_alpha2_to_country_name)
data_grouped["Country Fullname"] = data_grouped["Country Fullname"].str.split(',').str[0]


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")

In [ ]:
# Example
print(geolocator.geocode('Tanzania').longitude)

35.7878438


In [ ]:
# Geolocalize (longitude, latitude) each country 
data_grouped["GCode"] = data_grouped["Country Fullname"].apply(geolocator.geocode)

In [ ]:
# Extract Latitude and Longitude from GCode column
data_grouped['Latitude'] = [g.latitude for g in data_grouped["GCode"]]
data_grouped['Longitude'] = [g.longitude for g in data_grouped["GCode"]]
data_grouped

,Country,count,Country Fullname,GCode,Latitude,Longitude
0,US,309,United States,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
1,IN,9,India,"(India, (22.3511148, 78.6677428))",22.351115,78.667743
2,MX,4,Mexico,"(México, (22.5000485, -100.000037))",22.500048,-100.000037
3,AE,3,United Arab Emirates,"(الإمارات العربية المتحدة, (24.0002488, 53.999...",24.000249,53.999483
4,BE,2,Belgium,"(België / Belgique / Belgien, (50.6402809, 4.6...",50.640281,4.666715
5,KE,2,Kenya,"(Kenya, (1.4419683, 38.4313975))",1.441968,38.431398
6,SG,2,Singapore,"(Singapore, (1.357107, 103.8194992))",1.357107,103.819499
7,CA,2,Canada,"(Canada, (61.0666922, -107.991707))",61.066692,-107.991707
8,NL,2,Netherlands,"(Nederland, (52.15517, 5.38721))",52.155170,5.387210
9,LK,2,Sri Lanka,"(ශ්‍රී ලංකාව இலங்கை, (7.5554942, 80.7137847))",7.555494,80.713785


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Empty map
world_map= folium.Map(width=800, height=600, tiles="cartodbpositron")
# For each coordinate, create circlemarker of the number of mentors
for i in range(len(data_grouped)):
        lat = data_grouped.iloc[i]['Latitude']
        long = data_grouped.iloc[i]['Longitude']
        radius = max(data_grouped.iloc[i]['count']/10, 1)
        popup_text = "Number of mentors: {}"
        popup_text = popup_text.format(data_grouped.iloc[i]['count'])
        folium.CircleMarker(location=[lat, long], radius=radius, popup=popup_text, fill=True).add_to(world_map)

In [ ]:
world_map